interpretation for white studnets: grades increased in post-treatment

Chi2 statistical test

In [ ]:
from scipy.stats import chi2_contingency

In [ ]:
contingency_table = (
    df['m_curr_ALL'],
    df['treatment'])
_, p, _, _ = chi2_contingency(contingency_table, lambda_="log-likelihood")

# p-value
p
# how else to get p?

## moving on

In [ ]:
df.plot.scatter(x='total_gran', y='m_curr_ALL', c='treatment', cmap='rainbow', colorbar=True)

In [ ]:
df

In [ ]:
from causalinference import CausalModel
# CausalModel params:
# y = outcomes
# d = treatment
# X = covariates

[causalinference api](https://causalinferenceinpython.org/causalinference.html#module-causalinference.causal)

In [ ]:
cm = CausalModel(
    Y=df.m_curr_ALL.values,
    D=df.treatment.values,
    X=df.e_curr_ALL.values)

cm.est_via_ols(adj=0)

print(cm.estimates)

# problem because data not linear enough?

In [ ]:
cm = CausalModel(
    Y=df.m_curr_ALL.values, 
    D=df.treatment.values, 
    X=df.e_curr_ALL.values)

cm.est_via_matching()

print(cm.estimates)

In [ ]:
import statsmodels.formula.api as smf

In [ ]:
results = smf.ols(formula='m_curr_ALL ~ treatment', data=df,).fit()
print(results.summary())